In [ ]:
!pip install datasets --quiet
!pip install transformers --quiet
!pip install torch --quiet
!pip install sentencepiece --quiet
!pip install git+https://github.com/huggingface/accelerate --quiet

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import numpy as np 
import pandas as pd 
from tqdm import tqdm
import warnings
import os


warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

%cd /content/drive/Shareddrives/DL + NLP/Proyecto DL + NLP/dataset/RedditMHD/Data

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/.shortcut-targets-by-id/1j0RASnE3IntTbVJIRuwXAvt885-X9BEv/Proyecto DL + NLP/dataset/RedditMHD/Data


In [ ]:
%ls

 ADHD/                      combined.csv         PTSD/
 Anxiety/                   corpus_adhd.txt      Schizophrenia/
 Bipolar/                   corpus_anxiety.txt   test.csv
 BPD/                       DATASET_CONTROL/     test_trainer/
 combined_balanced.csv      Depression/          tokenizers/
'combined_balanced.csv '/   EDA/                 train.csv


In [ ]:
data = pd.read_csv('./combined_balanced.csv')

In [ ]:
data

,subreddit,post,label
0,control,What do you do when most of a class passes and...,0
1,control,"Picking a chapter book for ""cool down time"" I'...",0
2,control,How can I prevent alumni from visiting me? As ...,0
3,control,Why I feel like a dumb student In elementary S...,0
4,control,Anonymous questionnaire on Social Anxiety Hell...,0
...,...,...,...
143995,bipolarreddit,Business travel w/ bipolar? Hi Friends -\n\nI'...,3
143996,bipolarreddit,Finding a job when depressed is impossible. Fi...,3
143997,bipolarreddit,Mania?? I was diagnosed with bipolar over 4 ye...,3
143998,bipolarreddit,Are any of you successfully managing your cond...,3


In [ ]:
pattern_1 = "adhd"
pattern_2 = "anxiety"

filter_1 = data['subreddit'].str.contains(pattern_1)
filter_2 = data['subreddit'].str.contains(pattern_2)

data_1 = data[filter_1]
data_2 = data[filter_2]

In [ ]:
frames = [data_1, data_2]

data = pd.concat(frames)

In [ ]:
data

,subreddit,post,label
48000,adhd,Has anyone tried the GeneSight genetic test fo...,1
48001,adhd,I was doing so well... I was diagnosed back in...,1
48002,adhd,Is it possible to have ADHD and not typically ...,1
48003,adhd,"Is it possible to treat ADD, anxiety, and depr...",1
48004,adhd,Everything seems very complicated. I was diagn...,1
...,...,...,...
47995,anxiety,Anyone try TMS? I am starting TMS on Monday fr...,2
47996,anxiety,So I’ve been thinking of getting Propranolol f...,2
47997,anxiety,"Organic, natural, vegan CBD Hey guys, you've p...",2
47998,anxiety,Anxiety or MS? 25/M. Over the last four month...,2


In [ ]:
train_text_df  = data.rename(columns={'subreddit': 'entities', 'post': 'text', 'label': 'labels'})

In [ ]:
train_text_df ["id"] = train_text_df .index + 1

In [ ]:
train_text_df ['entities'] = train_text_df ['entities'].str.strip('()').str.split(',')

In [ ]:
from sklearn.utils import shuffle
train_text_df = shuffle(train_text_df)

In [ ]:
train_text_df.drop(['labels'], axis=1, inplace=True)

In [ ]:
train_text_df = train_text_df.reset_index(drop=True)

In [ ]:
train_text_df 

,entities,text,id
0,[anxiety],Built this anxiety battling app to try and hel...,42480
1,[anxiety],Been off work 2 weeks for xmas. I go back toda...,46156
2,[adhd],Why Am I So Annoying I cringe so hard at the t...,59342
3,[anxiety],what type of anxiety do I have? So I've been d...,47886
4,[anxiety],"Quit job, worried I can’t do a new one I quit ...",34044
...,...,...,...
31995,[anxiety],Is there a tie between having anxiety and feel...,45576
31996,[adhd],Need advice with maintaining networks I'm not ...,54388
31997,[adhd],Teen Psych Ed Eval before Psych Visit Need som...,60732
31998,[anxiety],Currently laying on the kitchen floor having a...,34258


In [ ]:
import ast

train_text_df['entities'] = train_text_df['entities'].apply(lambda x:ast.literal_eval(str(x)))

print(train_text_df['entities'].values[0])

['anxiety']


In [ ]:
train_text_df = train_text_df[:]

In [ ]:
import datasets
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding
from transformers import BigBirdForTokenClassification, BigBirdTokenizerFast
from torch import cuda
import torch

In [ ]:
config = {'model_name': 'google/bigbird-roberta-base',
         'max_length': 150,
         'train_batch_size':4,
         'valid_batch_size':4,
         'epochs':5,
         'learning_rate':5e-05,
         'max_grad_norm':10,
          'warmup':0.1,
          "grad_acc":8,
          "model_save_path":"big-bird",
         'device': 'cuda' if cuda.is_available() else 'cpu'}

output_labels = ['anxiety', 'adhd']

labels_to_ids = {v:k for k,v in enumerate(output_labels)}
ids_to_labels = {k:v for k,v in enumerate(output_labels)}

In [ ]:
train_text_df['labels'] = train_text_df['entities'].apply(lambda x: [labels_to_ids[i] for i in x])

In [ ]:
train_text_df.head()

,entities,text,id,labels
0,[anxiety],Built this anxiety battling app to try and hel...,42480,[0]
1,[anxiety],Been off work 2 weeks for xmas. I go back toda...,46156,[0]
2,[adhd],Why Am I So Annoying I cringe so hard at the t...,59342,[1]
3,[anxiety],what type of anxiety do I have? So I've been d...,47886,[0]
4,[anxiety],"Quit job, worried I can’t do a new one I quit ...",34044,[0]


In [ ]:
tokenizer = BigBirdTokenizerFast.from_pretrained(config['model_name'])
model = BigBirdForTokenClassification.from_pretrained(config['model_name'],
                                                     num_labels=len(output_labels))

Some weights of the model checkpoint at google/bigbird-roberta-base were not used when initializing BigBirdForTokenClassification: ['cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BigBirdForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BigBirdForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BigBirdForTokenClassification were no

In [ ]:
converted = tokenizer(train_text_df.loc[0].values[1].split(),
                      is_split_into_words=True, return_offsets_mapping=True)

In [ ]:
ix = 0
for i,j in zip(tokenizer.convert_ids_to_tokens(converted['input_ids']), converted['offset_mapping']):
    print(i, j)
    ix += 1
    if ix == 15:
        break

[CLS] (0, 0)
▁Built (0, 5)
▁this (0, 4)
▁anxiety (0, 7)
▁battling (0, 8)
▁app (0, 3)
▁to (0, 2)
▁try (0, 3)
▁and (0, 3)
▁help (0, 4)
▁us (0, 2)
▁x (0, 1)
▁Hey (0, 3)
▁everybody (0, 9)
! (9, 10)


In [ ]:
def tokenizer_data(example):
    encoding = tokenizer(example['text'].split(),
                         is_split_into_words=True,
                         truncation=True,
                         padding='max_length', 
                         return_offsets_mapping=True,
                         max_length=config['max_length'])
    i = 0
    labels = example['labels']
    encoded_labels = np.ones(len(encoding["offset_mapping"]), dtype=int) * -100
    for idx, mapping in enumerate(encoding["offset_mapping"]):
        if mapping[0] == 0 and mapping[1] != 0:
            try:
                encoded_labels[idx] = labels[i]
            except:
                pass
            i += 1
    item = {key: torch.as_tensor(val) for key, val in encoding.items()}
    item['labels'] = torch.as_tensor(encoded_labels)
    return item

In [ ]:
dataset = datasets.Dataset.from_pandas(train_text_df)

In [ ]:
dataset = dataset.train_test_split(test_size=0.1)
dataset

DatasetDict({
    train: Dataset({
        features: ['entities', 'text', 'id', 'labels'],
        num_rows: 28800
    })
    test: Dataset({
        features: ['entities', 'text', 'id', 'labels'],
        num_rows: 3200
    })
})

In [ ]:
text = dataset['train'][1]

In [ ]:
converted = tokenizer_data(text)

converted

{'input_ids': tensor([   65,  2662,   438,  5480, 10579,  2286,   131,   415,   100,   177,
           358,  1697,   108,   145,   109,   715,  1569,  1130,  1625,  3811,
           112,   391,  2146,   385,   568,   385,  4253,  1407,   715,   112,
           474,   358,  7919,  3811,   388,  1130,  1625,   938,  1413,  1523,
           100,   184,  2294,  2098,   391,   884,  1117,   757,  4253,   582,
           415,   408,  1599,   385,  7967,   452, 14204,   131,   415,  1917,
           385,   363, 31308,  7516,   391,   474, 11364,   712,   441,   474,
          2962,   441,   112,   718,   762,  1011,   441,   100,   183,   358,
         20921,  2664,   391,   718,   762,  1011,   441,  4131,   707,   363,
          1077,   114, 10448,   446,   578,  1410,   603,   861,   712,   386,
          5480,   490,  2962,   441,   131,   458,  1921,   613, 31399,   419,
          6150,   415,   561,   100,   184,   752,  2098,  1861,   109,    66,
             0,     0,     0,     0,   

In [ ]:
converted

i=0
for token, label in zip(tokenizer.convert_ids_to_tokens(converted["input_ids"]), 
                        converted["labels"]):
    print(token, label, converted['offset_mapping'][i])
    i+=1
    if i == 15:
        break

[CLS] tensor(-100) tensor([0, 0])
▁Will tensor(1) tensor([0, 4])
▁M tensor(-100) tensor([0, 1])
eds tensor(-100) tensor([1, 4])
▁Help tensor(-100) tensor([0, 4])
▁Me tensor(-100) tensor([0, 2])
? tensor(-100) tensor([2, 3])
▁I tensor(-100) tensor([0, 1])
<unk> tensor(-100) tensor([1, 2])
m tensor(-100) tensor([2, 3])
▁a tensor(-100) tensor([0, 1])
▁17 tensor(-100) tensor([0, 2])
( tensor(-100) tensor([2, 3])
M tensor(-100) tensor([3, 4])
) tensor(-100) tensor([4, 5])


In [ ]:
dataset = dataset.map(tokenizer_data)

dataset

Map:   0%|          | 0/28800 [00:00<?, ? examples/s]

Map:   0%|          | 0/3200 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['entities', 'text', 'id', 'labels', 'input_ids', 'attention_mask', 'offset_mapping'],
        num_rows: 28800
    })
    test: Dataset({
        features: ['entities', 'text', 'id', 'labels', 'input_ids', 'attention_mask', 'offset_mapping'],
        num_rows: 3200
    })
})

In [ ]:
dataset.set_format(type='torch', columns=['input_ids', 'attention_mask',
                                         'labels'])

dataset

DatasetDict({
    train: Dataset({
        features: ['entities', 'text', 'id', 'labels', 'input_ids', 'attention_mask', 'offset_mapping'],
        num_rows: 28800
    })
    test: Dataset({
        features: ['entities', 'text', 'id', 'labels', 'input_ids', 'attention_mask', 'offset_mapping'],
        num_rows: 3200
    })
})

In [ ]:
for a in dataset['train']:
    if a['input_ids'].shape[0] != a['attention_mask'].shape[0]:
        print(a)
        break

In [ ]:
%ls

 ADHD/                      combined.csv         PTSD/
 Anxiety/                   corpus_adhd.txt      Schizophrenia/
 Bipolar/                   corpus_anxiety.txt   test.csv
 BPD/                       DATASET_CONTROL/     test_trainer/
 combined_balanced.csv      Depression/          tokenizers/
'combined_balanced.csv '/   EDA/                 train.csv


In [ ]:
trainer_args = TrainingArguments(report_to='none',
                                output_dir="../BigBird",
                                num_train_epochs=config['epochs'],
                                evaluation_strategy ='epoch',
                                per_device_train_batch_size=config['train_batch_size'],
                                per_device_eval_batch_size=config['valid_batch_size'],
                                fp16=True,
                                save_strategy = "epoch",
                                warmup_ratio= config['warmup'],
                                gradient_accumulation_steps=config['grad_acc'],
                                logging_strategy="epoch",
                                save_total_limit=1
                                )

trainer = Trainer(model=model,
                  args=trainer_args, 
                  train_dataset = dataset['train'],
                  eval_dataset=dataset['test'],
                  tokenizer=tokenizer)

In [ ]:
trainer.train("../BigBird/checkpoint-2700")

You are resuming training from a checkpoint trained with 4.29.1 of Transformers but your current version is 4.29.2. This is not recommended and could yield to errors or unwanted behaviors.
You're using a BigBirdTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Attention type 'block_sparse' is not possible if sequence_length: 150 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 704 with config.block_size = 64, config.num_random_blocks = 3. Changing attention type to 'original_full'...


Epoch,Training Loss,Validation Loss
4,0.040900,0.028300
5,0.016900,0.028512


TrainOutput(global_step=4500, training_loss=0.011556512196858723, metrics={'train_runtime': 1124.8718, 'train_samples_per_second': 128.015, 'train_steps_per_second': 4.0, 'total_flos': 1.1099997648e+16, 'train_loss': 0.011556512196858723, 'epoch': 5.0})

In [ ]:
#trainer.train()

In [ ]:
device = config['device']

In [ ]:
trainer.model.eval()
def inference(sentence):
    inputs = tokenizer(sentence.split(),
                        is_split_into_words=True, 
                        return_offsets_mapping=True, 
                        padding='max_length', 
                        truncation=True, 
                        max_length=200,
                        return_tensors="pt")

    # move to gpu
    ids = inputs["input_ids"].to(device)
    mask = inputs["attention_mask"].to(device)
    # forward pass
    outputs = trainer.model(input_ids=ids, attention_mask=mask, return_dict=False)
#     print(outputs)
    logits = outputs[0]
    
    active_logits = logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
    flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size*seq_len,) - predictions at the token level
    print(logits.shape, active_logits.shape, flattened_predictions.shape)
    tokens = tokenizer.convert_ids_to_tokens(ids.squeeze().tolist())
    token_predictions = [ids_to_labels[i] for i in flattened_predictions.cpu().numpy()]
    wp_preds = list(zip(tokens, token_predictions)) # list of tuples. Each tuple = (wordpiece, prediction)

    prediction = []
    out_str = []
    off_list = inputs["offset_mapping"].squeeze().tolist()
    for idx, mapping in enumerate(off_list):
#         print(mapping, token_pred[1], token_pred[0],"####")

#         only predictions on first word pieces are important
        if mapping[0] == 0 and mapping[1] != 0:
#             print(mapping, token_pred[1], token_pred[0])
            prediction.append(wp_preds[idx][1])
            out_str.append(wp_preds[idx][0])
        else:
            if idx == 1:
                prediction.append(wp_preds[idx][1])
                out_str.append(wp_preds[idx][0])
            continue
    return prediction, out_str

In [ ]:
text_1= 'Ive never taken it to a doctor, but Ive heard that it could help for anxiety, but I dont want to take it to the point where I am constantly afraid of dying.'
pred_1, _ = inference(text_1)
print(pred_1[0])

torch.Size([1, 200, 2]) torch.Size([200, 2]) torch.Size([200])
anxiety


In [ ]:
text_2= 'I am not sure if this is a sign that I am not sick or if I am just lazy and have no sense of direction.'
pred_2, _ = inference(text_2)
print(pred_2[0])

torch.Size([1, 200, 2]) torch.Size([200, 2]) torch.Size([200])
adhd
